In [ ]:
def load_objects_from_json__ujson(self):
        start_time = time.time()
        listObj = [{'frame_idx': i + 1, 'frame_data': []}
                    for i in range(self.TOTAL_VIDEO_FRAMES)]
        json_file_name = f'{self.CURRENT_VIDEO_PATH}/{self.CURRENT_VIDEO_NAME}_tracking_results.json'
        if not os.path.exists(json_file_name):
            with open(json_file_name, 'w') as jf:
                ujson.dump(listObj, jf,
                            indent=4,
                            separators=(',', ': '))
            jf.close()
        with open(json_file_name, 'r') as jf:
            listObj = ujson.load(jf)
        jf.close()
        end_time = time.time()
        print(f"Time taken to load json with (ujson)library is : {int((end_time - start_time)*1000)} ms" + "\n")
        return listObj

def load_objects_to_json__ujson(self, listObj):
    start_time = time.time()
    json_file_name = f'{self.CURRENT_VIDEO_PATH}/{self.CURRENT_VIDEO_NAME}_tracking_results.json'
    with open(json_file_name, 'w') as jf:
        ujson.dump(listObj, jf,
                    indent=4,
                    separators=(',', ': '))
    jf.close()
    end_time = time.time()
    print(f"Time taken to write json with (ujson)library is : {int((end_time - start_time)*1000)} ms" + "\n")

def load_objects_from_json__rapidjson(self):
    start_time = time.time()
    listObj = [{'frame_idx': i + 1, 'frame_data': []}
                for i in range(self.TOTAL_VIDEO_FRAMES)]
    json_file_name = f'{self.CURRENT_VIDEO_PATH}/{self.CURRENT_VIDEO_NAME}_tracking_results.json'
    if not os.path.exists(json_file_name):
        with open(json_file_name, 'w') as jf:
            rapidjson.dump(listObj, jf,
                        indent=4,
                        separators=(',', ': '))
        jf.close()
    with open(json_file_name, 'r') as jf:
        listObj = rapidjson.load(jf)
    jf.close()
    end_time = time.time()
    print(f"Time taken to load json with (rapidjson)library is : {int((end_time - start_time)*1000)} ms" + "\n")
    return listObj

def load_objects_to_json__rapidjson(self, listObj):
    start_time = time.time()
    json_file_name = f'{self.CURRENT_VIDEO_PATH}/{self.CURRENT_VIDEO_NAME}_tracking_results.json'
    with open(json_file_name, 'w') as jf:
        rapidjson.dump(listObj, jf,
                    indent=4,
                    separators=(',', ': '))
    jf.close()
    end_time = time.time()
    print(f"Time taken to write json with (rapidjson)library is : {int((end_time - start_time)*1000)} ms" + "\n")
    
def load_objects_from_json__msgspec(self):
    start_time = time.time()
    listObj = [{'frame_idx': i + 1, 'frame_data': []}
                for i in range(self.TOTAL_VIDEO_FRAMES)]
    json_file_name = f'{self.CURRENT_VIDEO_PATH}/{self.CURRENT_VIDEO_NAME}_tracking_results.json'
    if not os.path.exists(json_file_name):
        with open(json_file_name, 'w') as jf:
            jf.write(msgspec.json.encode(listObj))
        jf.close()
    with open(json_file_name, 'r') as jf:
        listObj = msgspec.json.decode(jf.read())
    jf.close()
    end_time = time.time()
    print(f"Time taken to load json with (msgspec)library is : {int((end_time - start_time)*1000)} ms" + "\n")
    return listObj

def load_objects_to_json__msgspec(self, listObj):
    start_time = time.time()
    json_file_name = f'{self.CURRENT_VIDEO_PATH}/{self.CURRENT_VIDEO_NAME}_tracking_results.json'
    with open(json_file_name, 'w') as jf:
        jf.write(str(msgspec.json.encode(listObj)))
    jf.close()
    end_time = time.time()
    print(f"Time taken to write json with (msgspec)library is : {int((end_time - start_time)*1000)} ms" + "\n")
    
def RUNALLJSONS(self):
        
        print("\n\nRUNNING ALL JSONS\n")
        listObj = self.load_objects_from_json__json()
        self.load_objects_from_json__orjson()
        self.load_objects_from_json__ujson()
        self.load_objects_from_json__msgspec()
        self.load_objects_from_json__rapidjson()
        
        print("\n\nRUNNING ALL JSONS\n")
        self.load_objects_to_json__json(listObj)
        self.load_objects_to_json__orjson(listObj)
        self.load_objects_to_json__ujson(listObj)
        #self.load_objects_to_json__msgspec(listObj)
        #self.load_objects_to_json__rapidjson(listObj)
        

In [14]:
dictObj = {'id_1' : [1,2,3,4,5,6,7,8,9,10],
              'id_22' : [11,12,13,14,15,16,17,18,19,20],
                'id_322' : [21,22,23,24,25,26,27,28,29,30],
                    'id_4222' : [31,32,33,34,35,36,37,38,39,40],
                        'id_522222' : [41,42,43,44,45,46,47,48,49,50],}
keys = list(dictObj.keys())
ids = [int(keys[i][3:]) for i in range(len(keys))]
print(ids)

[1, 22, 322, 4222, 522222]


In [17]:
x = set()
min(x)

ValueError: min() arg is an empty sequence